In [ ]:
import os

import ipywidgets as ipy
import pandas as pd
from sklearn.externals import joblib
from skimage.color import label2rgb

DATA_DIR = '/public/dsb_2018_data'
PREDICTION_TRAIN = 'predictions/unet/train/outputs'
PREDICTION_VALID = 'predictions/unet/valid/outputs'
PREDICTION_TEST = 'predictions/unet/test/outputs'

SUBMISSION_PATH = os.path.join('/output/submission.csv')

In [ ]:
meta = pd.read_csv(os.path.join(DATA_DIR,'stage1_metadata.csv'))
meta_train = meta[meta['is_train']==1 & (~meta['vgg_features_clusters'].isin([0,1]))]
meta_valid = meta[meta['is_train']==1 & (meta['vgg_features_clusters'].isin([0,1]))]
meta_test = meta[meta['is_train']==0]

In [ ]:
def plot_with_labels(mask, contour, postprocessed, label, idx):
    plt.figure(figsize=(16,12))
    plt.subplot(141)
    plt.imshow(mask[idx])
    plt.subplot(142)
    plt.imshow(contour[idx])
    plt.subplot(143)
    plt.imshow(label2rgb(postprocessed[idx]))
    plt.subplot(144)
    plt.imshow(label2rgb(label[idx]))
    plt.show()
    
def plot_predictions(mask, contour, postprocessed, idx):
    plt.figure(figsize=(16,12))
    plt.subplot(131)
    plt.imshow(mask[idx])
    plt.subplot(132)
    plt.imshow(contour[idx])
    plt.subplot(133)
    plt.imshow(label2rgb(postprocessed[idx]))
    plt.show()

# Predictions and labels on train

In [ ]:
labels_train = joblib.load(os.path.join(DATA_DIR,'ground_truth','train','labels.pkl'))
masks_train = joblib.load(os.path.join(DATA_DIR,PREDICTION_TRAIN,'unet_multitask'))['mask_prediction']
contours_train = joblib.load(os.path.join(DATA_DIR,PREDICTION_TRAIN,'unet_multitask'))['contour_prediction']
postprocessed_train = joblib.load(os.path.join(DATA_DIR,PREDICTION_TRAIN,'binary_fill'))['filled_images']

In [ ]:
ipy.interact(plot_with_labels, mask = ipy.fixed(masks_train),
              contour = ipy.fixed(contours_train),
              postprocessed = ipy.fixed(postprocessed_train),
              label = ipy.fixed(labels_train),
              idx = ipy.IntSlider(min=0, max=50, value=0, step=1))

# Predictions and labels on valid

In [ ]:
labels_valid = joblib.load(os.path.join(DATA_DIR,'ground_truth','valid','labels.pkl'))
masks_valid = joblib.load(os.path.join(DATA_DIR,PREDICTION_VALID,'unet_multitask'))['mask_prediction']
contours_valid = joblib.load(os.path.join(DATA_DIR,PREDICTION_VALID,'unet_multitask'))['contour_prediction']
postprocessed_valid = joblib.load(os.path.join(DATA_DIR,PREDICTION_VALID,'binary_fill'))['filled_images']

In [ ]:
ipy.interact(plot_with_labels, mask = ipy.fixed(masks_valid),
              contour = ipy.fixed(contours_valid),
              postprocessed = ipy.fixed(postprocessed_valid),
              label = ipy.fixed(labels_valid),
              idx = ipy.IntSlider(min=0, max=50, value=0, step=1))

# Predictions and submission on test

In [ ]:
masks_test = joblib.load(os.path.join(DATA_DIR,PREDICTION_TEST,'unet_multitask'))['mask_prediction']
contours_test = joblib.load(os.path.join(DATA_DIR,PREDICTION_TEST,'unet_multitask'))['contour_prediction']
postprocessed_test = joblib.load(os.path.join(DATA_DIR,PREDICTION_TEST,'binary_fill'))['filled_images']

In [ ]:
ipy.interact(plot_predictions, mask = ipy.fixed(masks_test),
              contour = ipy.fixed(contours_test),
              postprocessed = ipy.fixed(postprocessed_test),
              idx = ipy.IntSlider(min=0, max=50, value=0, step=1))

In [ ]:
def decompose(labeled):

    nr_true = labeled.max()
    masks = []
    for i in range(1, nr_true + 1):
        msk = labeled.copy()
        msk[msk != i] = 0.
        msk[msk == i] = 255.
        masks.append(msk)

    if not masks:
        return [labeled]
    else:
        return masks

def run_length_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0] 
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev + 1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths
    
def generate_submission(predictions, meta):
    image_ids, encodings = [], []
    for image_id, prediction in zip(meta['ImageId'].values, predictions):
        for mask in decompose(prediction):
            image_ids.append(image_id)
            encodings.append(' '.join(str(rle) for rle in run_length_encoding(mask > 128.)))

    submission = pd.DataFrame({'ImageId': image_ids, 'EncodedPixels': encodings})
    return submission

In [ ]:
submission = generate_submission(postprocessed_test, meta_test)
submission.head()

In [ ]:
submission.to_csv(SUBMISSION_PATH)